# Start Daemons
* reduce cold start latency by running servers

In [ ]:
import os
import subprocess
import time

In [ ]:
procs = []
source_to_port = {"mal": 3000, "anilist": 3001, "kitsu": 3002, "animeplanet": 3003}
python_script_to_port = {
    "Transformer": 3004,
    "BagOfWords": 3005,
}
julia_script_to_port = {
    "CompressSplits.jl": 3006,
    "Baseline.jl": 3007,
    "BagOfWords.jl": 3008,
    "Nondirectional.jl": 3009,
    "Transformer.jl": 3010,
    "Ensemble.jl": 3011,
    "Recommendations.jl": 3012,
}

In [ ]:
def runcmd(cmdlist, env=None):
    procs.append(
        subprocess.Popen(
            cmdlist, env=env, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT
        )
    )

In [ ]:
def juliacmd(cmdlist):
    port = julia_script_to_port[cmdlist[-1]]
    cmdlist.append(str(port))
    runcmd(cmdlist)

In [ ]:
for source, port in source_to_port.items():
    env = os.environ.copy()
    env["RSYS_LIST_SOURCE"] = source
    runcmd(
        [
            "gunicorn",
            "-w",
            "2",
            "FetchMediaLists:app",
            "-b",
            f"localhost:{port}",
        ],
        env,
    )

In [ ]:
juliacmd(["julia", "-t", "4", "Recommendations.jl"])

In [ ]:
os.chdir("../InferenceAlphas")

In [ ]:
for server, workers in zip(
    [
        "CompressSplits.jl",
        "Baseline.jl",
        "BagOfWords.jl",
        "Nondirectional.jl",
        "Transformer.jl",
        "Ensemble.jl",
    ],
    [2, 2, 8, 2, 2, 4],
):
    juliacmd(["julia", "-t", str(workers), server])

In [ ]:
for script, workers in zip(["Transformer", "BagOfWords"], [2, 8]):
    runcmd(
        [
            "gunicorn",
            "-w",
            str(workers),
            f"{script}:app",
            "-b",
            f"localhost:{python_script_to_port[script]}",
        ]
    )

In [ ]:
# keep the notebook alive to keep the procs alive
while True:
    time.sleep(3600)

In [ ]:
for p in procs:
    p.terminate()